# Pre-processing the raw enron email data

1. Transform unix time to datetime
2. Filter sender/recipients data
3. Extract Person's name from email address
4. Generate count dataframe saved in result/output.csv



## Import Packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import json
import datetime
from datetime import datetime as dt


color = sns.color_palette()

%matplotlib inline
print "Packages Loaded Successfully"

Packages Loaded Successfully


## Helper Functions

In [2]:
## Execution time helper function

class timeit():
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

        
## Split list in columns to multiple rows

def split_list_in_cols_to_rows(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

## Load Data

In [3]:
All_df = pd.read_csv('data/enron-event-history-all.csv',
                     names = ['time','message','sender','recipients','topic','mode'])

print("Number of rows: {:.0f}".format(All_df.shape[0]))

Number of rows: 205731


### check column object

In [4]:
All_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205731 entries, 0 to 205730
Data columns (total 6 columns):
time          205731 non-null int64
message       205731 non-null object
sender        205699 non-null object
recipients    205693 non-null object
topic         0 non-null float64
mode          205731 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 9.4+ MB


### Check Null Values

In [5]:
null_counts = All_df.isnull().sum()
print(null_counts)

time               0
message            0
sender            32
recipients        38
topic         205731
mode               0
dtype: int64


### Check Mode's unique value

In [6]:
All_df['mode'].value_counts()

email    205731
Name: mode, dtype: int64

### Drop Topic, Mode, and Message Columns

In [7]:
All_df.drop(['topic','mode','message'],axis=1,inplace=True)

### Drop null rows

In [8]:
All_df.dropna(how='any',inplace=True)

## Transform unix time to datetime


In [10]:
## Transform unix time to datetime

All_df['time'] = pd.to_datetime(All_df['time'],unit='ms')



## Filter sender/recipients data

### Check Sender value count

In [13]:
All_df.sender.value_counts()[:30]

jeff dasovich        5232
sara shackleton      4591
pete davis           3898
chris germany        3703
notes                3314
vince kaminski       2937
matthew lenhart      2832
debra perlingiere    2611
gerald nemec         2353
announcements        2243
mark taylor          2110
richard sanders      1920
james steffes        1748
john arnold          1710
michelle cash        1639
kate symes           1636
blank                1603
sally beck           1592
elizabeth sager      1554
phillip love         1484
Carol St Clair       1447
louise kitchen       1443
darron giron         1261
outlook              1160
drew fossum          1147
benjamin rogers      1114
kevin presto         1108
mark haedicke        1101
chris dorland        1082
jeffrey shankman     1054
Name: sender, dtype: int64

* Notes/blank/outlook doesn't look like name.
* Notes seems like @everyone group message
* blank/outlook seems like unknown sender name

### Check data that sender is  notes

In [14]:
## Sort dataframe by time to see time series pattern
All_df.sort_values(by='time',inplace=True)

All_df.loc[All_df['sender'] =='notes'].head(20)

,time,sender,recipients
67833,2001-03-07 22:42:00,notes,juan hernandez
69968,2001-03-15 20:10:00,notes,juan hernandez
74543,2001-04-02 08:41:00,notes,mary hain
77075,2001-04-10 00:40:32,notes,avallow lodielectric|angles cook inlet
78092,2001-04-11 23:53:20,notes,avallow lodielectric|anglea cook inlet
82075,2001-04-24 04:16:00,notes,pkeavey
82254,2001-04-24 16:16:00,notes,pkeavey
82380,2001-04-24 19:23:00,notes,pkeavey
83486,2001-04-25 07:23:00,notes,pkeavey
84399,2001-04-27 02:13:05,notes,gworster txu|jcattermole pcenergy|eric gillasp...


* only one recipients for each entity. So it's not group message
* replace it as unknown sender name 


### Replace notes/blank/outlook sender as unknown sender

In [15]:
All_df.sender.replace(['notes', 'blank','outlook'], 'unknown', inplace=True)


In [16]:
## print number of unknown sender records

count = All_df.loc[All_df['sender'] =='unknown'].shape[0]
print("Number of unknown sender: {:.0f}".format(count))

Number of unknown sender: 6077


### Check Sender value count

In [17]:
All_df.recipients.value_counts()[:30]

sara shackleton                                      2340
bert meyers|bill williams|craig dean|geir solberg    2232
kenneth lay                                          1883
louise kitchen                                       1759
gerald nemec                                         1665
sally beck                                           1319
vince kaminski                                       1242
kate symes                                           1118
mark taylor                                          1105
andrew lewis                                         1071
daren farmer                                          881
pete davis|bert meyers|bill williams                  845
mike maggi                                            842
kim ward                                              801
jeff dasovich                                         779
john lavorato                                         775
richard shapiro                                       747
benjamin roger

* one entity can have multiple recipients
* recipient has notes as well

### Split multiple recipients 

In [18]:
with timeit():
    All_df = split_list_in_cols_to_rows(All_df.assign(recipients=All_df.recipients.str.split('|')), 'recipients')

runtime: 0:00:00.682795


### Check overlap betwee email name and  person name in sender/recipients

In [21]:
All_df.head(20)

,time,sender,recipients
0,1998-05-27 22:31:00,Christopher Behney,Toni P Schulenburg
1,1998-05-27 22:31:00,Christopher Behney,mary hain
2,1998-10-30 23:43:00,mark legal taylor,Marc.R.Cutler@BankAmerica.com
3,1998-10-30 23:56:00,mark legal taylor,Marc.R.Cutler@BankAmerica.com
4,1998-10-31 00:02:00,mark legal taylor,shari stack
5,1998-10-31 01:06:00,mark legal taylor,Marc.R.Cutler@BankAmerica.com
6,1998-11-03 03:32:00,mark legal taylor,yao apasu
7,1998-11-03 19:12:00,yao apasu,brent hendry
8,1998-11-03 19:12:00,yao apasu,mark legal taylor
9,1998-11-04 18:31:00,mark legal taylor,Marc.R.Cutler@BankAmerica.com


In [22]:
## Check couple cases

## Check name in the Heather J Mitchell AT ENRON_DEVELOPMENT@CCMAIL email
print All_df.loc[All_df['recipients']=='Heather J Mitchell']
print '\n'

## Check name in Mark - ECT Legal Taylor@ECT
print All_df.loc[All_df['recipients']=='Mark - ECT Legal Taylor']
print '\n'

## Check name in the Marc.R.Cutler@BankAmerica.com
print All_df.loc[All_df['recipients']=='Marc.R.Cutler']


Empty DataFrame
Columns: [time, sender, recipients]
Index: []


Empty DataFrame
Columns: [time, sender, recipients]
Index: []


Empty DataFrame
Columns: [time, sender, recipients]
Index: []


* Seems no overlap betweent the Email Address and name

## Extract the name from the email address

In [23]:

All_df.recipients = All_df.recipients.apply(lambda x: x.split("@")[0])
All_df.sender = All_df.sender.apply(lambda x: x.split("@")[0])


### Check the pre-process df results

In [24]:
## First 10 rows in dataframe before pre-processing

raw_df = pd.read_csv('data/enron-event-history-all.csv',
                     names = ['time','message','sender','recipients','topic','mode'],
                     nrows=10)

raw_df[['time','sender','recipients']]

,time,sender,recipients
0,896308260000,Christopher Behney,Toni P Schulenburg|mary hain
1,909790980000,mark legal taylor,Marc.R.Cutler@BankAmerica.com
2,909791760000,mark legal taylor,Marc.R.Cutler@BankAmerica.com
3,909792120000,mark legal taylor,shari stack
4,909795960000,mark legal taylor,Marc.R.Cutler@BankAmerica.com
5,910063920000,mark legal taylor,yao apasu
6,910120320000,yao apasu,brent hendry|mark legal taylor
7,910204260000,mark legal taylor,Marc.R.Cutler@BankAmerica.com
8,910209540000,mark legal taylor,paul simons
9,910227780000,mark legal taylor,Cynthia C Sandherr@Enron|Jeffrey Keeler@Enron|...


In [25]:
## First 10 rows in dataframe after pre-processing

All_df.head(10)

,time,sender,recipients
0,1998-05-27 22:31:00,Christopher Behney,Toni P Schulenburg
1,1998-05-27 22:31:00,Christopher Behney,mary hain
2,1998-10-30 23:43:00,mark legal taylor,Marc.R.Cutler
3,1998-10-30 23:56:00,mark legal taylor,Marc.R.Cutler
4,1998-10-31 00:02:00,mark legal taylor,shari stack
5,1998-10-31 01:06:00,mark legal taylor,Marc.R.Cutler
6,1998-11-03 03:32:00,mark legal taylor,yao apasu
7,1998-11-03 19:12:00,yao apasu,brent hendry
8,1998-11-03 19:12:00,yao apasu,mark legal taylor
9,1998-11-04 18:31:00,mark legal taylor,Marc.R.Cutler


### Save the pre-processed dataframe into csv file

In [26]:
All_df.to_csv('data/cleaned.csv',index = False)

## Generate count dataframe saved in result

In [27]:
sender_list = All_df.sender.unique()
recipients_list = All_df.recipients.unique()
name_list = set(sender_list)
name_list = name_list.union(recipients_list)

print("No. of unique name in sender column : {:.0f}".format(len(sender_list)))
print("No. of unique name in recipient column : {:.0f}".format(len(recipients_list)))
print("No. of unique name in name list : {:.0f}".format(len(name_list)))

No. of unique name in sender column : 15211
No. of unique name in recipient column : 28737
No. of unique name in name list : 36372


### Initial result dataframe

In [28]:
result_df = pd.DataFrame(data = list(name_list),columns = ['person'])


### Hash the count result into result_df

In [29]:

## Count No. of emails sent for each person
sent_count_df = pd.DataFrame(All_df.groupby('sender').count()['time'].reset_index())
sent_count_df.columns=['person','sent']

## Count No. of emails received for each person
receive_count_df = pd.DataFrame(All_df.groupby('recipients').count()['time'].reset_index())
receive_count_df.columns=['person','received']


### Merge two count dataframe into result dataframe

In [30]:
result_df = pd.merge(result_df,sent_count_df,on = ['person'],how = 'left')
result_df = pd.merge(result_df,receive_count_df,on = ['person'],how = 'left')

### Sort it by number of emails sent

In [31]:
result_df.sort_values(by='sent',ascending=False,inplace=True)

### Fillna

In [32]:
result_df.fillna(0,inplace=True)

### Save it to Output1.csv

In [33]:
result_df.to_csv('result/Output11.csv',index=False)

## Finally, put everything together into a python function

* See lib/pre-processing.py